# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [135]:
import pandas as pd
import numpy as np
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [136]:
# 2) seu código aqui
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [137]:
# 3) seu código aqui
colunas = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_reduzido = sinasc.loc[:,colunas]
sinasc_reduzido.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [138]:
sinasc_reduzido.shape

(27028, 9)

In [139]:
# 4) seu código aqui
sinasc_reduzido = sinasc_reduzido[~sinasc_reduzido['APGAR5'].isna()]

In [140]:
sinasc_reduzido.describe()

,LOCNASC,IDADEMAE,ESTCIVMAE,QTDFILVIVO,CONSULTAS,APGAR5
count,26925.000000,26925.000000,26610.000000,25359.000000,26925.000000,26925.000000
mean,1.007168,26.095339,2.544720,1.050712,3.578793,9.179164
std,0.126025,6.386409,1.670264,1.166966,0.785540,0.790574
min,1.000000,11.000000,1.000000,0.000000,1.000000,0.000000
25%,1.000000,21.000000,1.000000,0.000000,3.000000,9.000000
50%,1.000000,26.000000,2.000000,1.000000,4.000000,9.000000
75%,1.000000,31.000000,5.000000,2.000000,4.000000,10.000000
max,4.000000,53.000000,9.000000,30.000000,9.000000,10.000000


In [141]:
sinasc_reduzido.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [142]:
# 5) seu código aqui
sinasc_reduzido['ESTCIVMAE'] = sinasc_reduzido['ESTCIVMAE'].fillna(9)
sinasc_reduzido['CONSULTAS'] = sinasc_reduzido['CONSULTAS'].fillna(9)

In [143]:
# 6) Seu código aqui
sinasc_reduzido['QTDFILVIVO'] = sinasc_reduzido['QTDFILVIVO'].fillna(0)

In [144]:
# 7) seu código aqui
percentagem = (sinasc_reduzido.isna().sum() / len(sinasc_reduzido)) * 100
percentagem

LOCNASC       0.000000
IDADEMAE      0.000000
ESTCIVMAE     0.000000
ESCMAE        1.151346
QTDFILVIVO    0.000000
GESTACAO      4.516249
GRAVIDEZ      0.282266
CONSULTAS     0.000000
APGAR5        0.000000
dtype: float64

Os dados faltantes da coluna ['GRAVIDEZ'] representam menos de 0.3% das linhas dos dados que estamos analisando.
O impacto que essa lacuna causará no resultado final da analise do DataFrame será tão irrisório que não justificaria
alocar tempo na busca da melhor forma de preencher esses espaços vazios. Sendo assim, optei por não
fazer qualquer tipo de alteração, afim de otimizar o tempo investido no projeto.

In [145]:
sinasc_reduzido['GESTACAO'].value_counts()

GESTACAO
37 a 41 semanas        22523
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: count, dtype: int64

In [146]:
percentagem_top_freq = ((sinasc_reduzido['GESTACAO'] == '37 a 41 semanas').sum() / sinasc_reduzido['GESTACAO'].count()) * 100
percentagem_top_freq

87.60745264304329

Na coluna ['GESTACAO'] é possível perceber que o valor '37 a 41 semanas' aparece em mais de 87% dos casos.
Se descartamos todas as linhas com dados faltantes, perderiamos 4.5% do nosso banco de dados,  porém, se preenchermos
os dados faltantes pelo valor mais frequente, podemos supor que cerca de 87% desses dados seriam preenchidos com o valor correto, o que nos deixaria com aproximadamente apenas 0.5% dos dados sendo 'não confiaveis'. (13% de 4.5% = 0.58%)
Então temos duas opções: perder 4.5% de todo o banco de dados OU não perder dados e aceitar que 0.5% deles não são confiavéis.
Por este motivo optei por preencher os dados faltantes com o valor de maior frequencia.

In [147]:
sinasc_reduzido['GESTACAO'] = sinasc_reduzido['GESTACAO'].fillna('37 a 41 semanas')
sinasc_reduzido['GESTACAO'].value_counts()

GESTACAO
37 a 41 semanas        23739
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: count, dtype: int64

7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.

8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:

Entre 8 e 10 está em uma faixa 'normal'.
Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
Entre 4 e 5 significa 'asfixia moderada'.
Entre 0 e 3 significa 'asfixia severa'.
Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.

9. Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras. Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True.

In [148]:
# 8) seu código aqui
sinasc_reduzido['ASFIXIA'] = pd.cut(sinasc_reduzido['APGAR5'], bins=[-1, 3, 5, 7, 10], labels=['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal'])

frequencias = sinasc_reduzido['ASFIXIA'].value_counts()

frequencias

ASFIXIA
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64

In [149]:
# 9) seu código aqui
sinasc_reduzido.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)
sinasc_reduzido

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,asfixia
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal


In [150]:
def para_snake_case(valor):
    if isinstance(valor, str):
        return valor.lower().replace(' ', '_')
    else:
        return valor


sinasc_reduzido = sinasc_reduzido.applymap(para_snake_case)
sinasc_reduzido

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,asfixia
0,1,19,5.0,8_a_11_anos,0.0,37_a_41_semanas,única,4,10.0,normal
1,1,29,2.0,8_a_11_anos,1.0,37_a_41_semanas,única,4,9.0,normal
2,1,37,9.0,8_a_11_anos,2.0,37_a_41_semanas,única,4,10.0,normal
3,1,30,5.0,12_anos_ou_mais,0.0,37_a_41_semanas,única,3,10.0,normal
4,1,30,2.0,8_a_11_anos,1.0,37_a_41_semanas,única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12_anos_ou_mais,1.0,32_a_36_semanas,única,4,9.0,normal
27024,1,19,1.0,8_a_11_anos,0.0,37_a_41_semanas,única,4,9.0,normal
27025,1,24,2.0,8_a_11_anos,0.0,37_a_41_semanas,única,4,10.0,normal
27026,1,21,2.0,8_a_11_anos,1.0,32_a_36_semanas,única,4,9.0,normal
